In [31]:
import pandas as pd
import os
import numpy as np

### Carregamento e Inspeção Inicial dos Dados

In [ ]:
# Carregar o dataset
df = pd.read_csv('dados_marketing.csv', sep=';')

# Verificar nulos
print(df.isnull().sum())

ID                                 0
Ano Nascimento                     0
Escolaridade                       0
Estado Civil                       0
Salario Anual                     19
Filhos em Casa                     0
Adolescentes em Casa               0
Data Cadastro                      0
Dias Desde Ultima Compra           0
Gasto com Eletronicos              0
Gasto com Brinquedos               0
Gasto com Moveis                   0
Gasto com Utilidades               0
Gasto com Alimentos                0
Gasto com Vestuario                0
Numero de Compras com Desconto     0
Numero de Compras na Web           0
Numero de Compras via Catalogo     0
Numero de Compras na Loja          0
Numero Visitas WebSite Mes         0
Compra na Campanha 1               0
Compra na Campanha 2               0
Compra na Campanha 3               0
Compra na Campanha 4               0
Compra na Campanha 5               0
Comprou                            0
Pais                               0
d

### Tratamento de Valores Nulos

In [33]:
# Análise dos registros com 'Salario Anual' nulo
salarios_nulos = df[df['Salario Anual'].isnull()]
print(pd.DataFrame(salarios_nulos['ID']).rename(columns={'ID': 'ID com Salário Nulo'}))

      ID com Salário Nulo
134                  8996
262                  1994
394                  3769
449                  5255
525                  8268
590                 10629
899                 10475
997                  9235
1185                 7187
1312                 8557
1515                 2863
1558                 2437
1693                 5250
1804                 7281
1858                 1612
1863                 5079
1880                10339
1967                 5798
1983                 2902


In [34]:
# Remoção dos nulos
df_inicial = df.shape[0]
df = df.dropna(subset=['Salario Anual'])
df_sem_nulos = df.shape[0]
print(f'Foram removidas {df_inicial - df_sem_nulos} linhas com salário nulo.')

Foram removidas 19 linhas com salário nulo.


### Tratamento de Duplicatas

In [35]:
# Remoção de duplicatas
duplicatas = df[df.duplicated()]
soma_duplicadas = df.duplicated().sum()
df = df.drop_duplicates()
print(f'Foram removidas {soma_duplicadas} linhas duplicadas.')

Foram removidas 0 linhas duplicadas.


### Engenharia de Atributos

In [36]:
# Conversão de datas
df['Data Cadastro'] = pd.to_datetime(df['Data Cadastro'], dayfirst=True)

In [37]:
# Criação de atributos derivados da Data de Cadastro
df['Ano Cadastro'] = df['Data Cadastro'].dt.year
df['Mes Cadastro'] = df['Data Cadastro'].dt.month
df['Mes Nome Cadastro'] = df['Data Cadastro'].dt.month_name()
df['Dia Cadastro'] = df['Data Cadastro'].dt.day
df['Dia Semana Cadastro'] = df['Data Cadastro'].dt.day_name()
df['Semana Cadastro'] = df['Data Cadastro'].dt.isocalendar().week

df['Idade'] = 2025 - df['Ano Nascimento']

In [38]:
# Criação de atributos de segmentação de idade
df['Faixa Etária'] = pd.cut(df['Idade'], bins=[17,30,45,60,100], labels=['18-30', '31-45', '46-60', '60+'])

In [39]:
# Criação de atributo binário para presença ou não de filhos/adolescentes
df['Filhos'] = np.where(df['Filhos em Casa'] > 0, 1, 0) | np.where(df['Adolescentes em Casa'] > 0, 1, 0)

### Criação de Métricas Derivadas

In [40]:
# Criação de métricas derivadas de gastos
gastos = ['Gasto com Eletronicos', 'Gasto com Brinquedos', 'Gasto com Moveis',
          'Gasto com Utilidades', 'Gasto com Alimentos', 'Gasto com Vestuario']
df['Gasto Total'] = df[gastos].sum(axis=1)
compras = ['Numero de Compras com Desconto', 'Numero de Compras na Web',
           'Numero de Compras via Catalogo', 'Numero de Compras na Loja']
df['Total Compras'] = df[compras].sum(axis=1)

In [41]:
# Criação de métricas derivadas de campanhas
df['Comprou em Campanha'] = np.where(df[[c for c in df.columns if 'campanha' in c.lower()]].sum(axis=1) > 0, 1, 0)
df['Engajamento em campanhas'] = df[[c for c in df.columns if 'campanha' in c.lower()]].sum(axis=1)

In [42]:
# Descrição do dataframe após a inclusão de novas colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1981 entries, 0 to 1999
Data columns (total 40 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   ID                              1981 non-null   int64         
 1   Ano Nascimento                  1981 non-null   int64         
 2   Escolaridade                    1981 non-null   object        
 3   Estado Civil                    1981 non-null   object        
 4   Salario Anual                   1981 non-null   float64       
 5   Filhos em Casa                  1981 non-null   int64         
 6   Adolescentes em Casa            1981 non-null   int64         
 7   Data Cadastro                   1981 non-null   datetime64[ns]
 8   Dias Desde Ultima Compra        1981 non-null   int64         
 9   Gasto com Eletronicos           1981 non-null   int64         
 10  Gasto com Brinquedos            1981 non-null   int64         
 11  Gasto com

### Padronização e Conversão de Tipos de Dados

In [43]:
# Função para padronizar tipos de dados
def padronizar_tipos_numericos(df):
    # Colunas que devem ser float64
    colunas_float = [
        'Salario Anual',
        'Gasto com Eletronicos',
        'Gasto com Brinquedos',
        'Gasto com Moveis',
        'Gasto com Utilidades',
        'Gasto com Alimentos',
        'Gasto com Vestuario',
        'Gasto Total'
    ]

    # Colunas que devem ser int64
    colunas_int = [
        'ID',
        'Ano Nascimento',
        'Filhos em Casa',
        'Adolescentes em Casa',
        'Dias Desde Ultima Compra',
        'Numero de Compras com Desconto',
        'Numero de Compras na Web',
        'Numero de Compras via Catalogo',
        'Numero de Compras na Loja',
        'Numero Visitas WebSite Mes',
        'Compra na Campanha 1',
        'Compra na Campanha 2',
        'Compra na Campanha 3',
        'Compra na Campanha 4',
        'Compra na Campanha 5',
        'Comprou',
        'Ano Cadastro',
        'Mes Cadastro',
        'Dia Cadastro',
        'Semana Cadastro',
        'Idade',
        'Filhos',
        'Total Compras',
        'Comprou em Campanha',
        'Engajamento em campanhas'
    ]

    # Aplica conversão
    for col in colunas_float:
        df[col] = df[col].astype('float64')

    for col in colunas_int:
        df[col] = df[col].astype('int64')

    return df


In [44]:
# Chama a função para efetivar a padronização
df = padronizar_tipos_numericos(df)

In [45]:
# Conversão de colunas binárias (valores 0 e 1) para Sim/Não
colunas_binarias = [col for col in df.columns
                    if df[col].dtype == 'int64' and set(df[col].unique()) <= {0, 1}]

# Converte para Sim/Não
for col in colunas_binarias:
    df[col] = df[col].map({1: 'Sim', 0: 'Não'})

### Padronização de Nomes de Colunas

In [46]:
# Padronização de nomes de colunas
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('ã', 'a').str.replace('ç', 'c').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [47]:
# Verificação da padronização de colunas
print(df.columns.tolist())

['id', 'ano_nascimento', 'escolaridade', 'estado_civil', 'salario_anual', 'filhos_em_casa', 'adolescentes_em_casa', 'data_cadastro', 'dias_desde_ultima_compra', 'gasto_com_eletronicos', 'gasto_com_brinquedos', 'gasto_com_moveis', 'gasto_com_utilidades', 'gasto_com_alimentos', 'gasto_com_vestuario', 'numero_de_compras_com_desconto', 'numero_de_compras_na_web', 'numero_de_compras_via_catalogo', 'numero_de_compras_na_loja', 'numero_visitas_website_mes', 'compra_na_campanha_1', 'compra_na_campanha_2', 'compra_na_campanha_3', 'compra_na_campanha_4', 'compra_na_campanha_5', 'comprou', 'pais', 'ano_cadastro', 'mes_cadastro', 'mes_nome_cadastro', 'dia_cadastro', 'dia_semana_cadastro', 'semana_cadastro', 'idade', 'faixa_etária', 'filhos', 'gasto_total', 'total_compras', 'comprou_em_campanha', 'engajamento_em_campanhas']


### Tradução de Dias e Meses (inglês > português)

In [48]:
# Mapeamento de dias da semana do inglês para português
dias_semana = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

# Aplica a tradução
df['dia_semana_cadastro'] = df['dia_semana_cadastro'].map(dias_semana)

In [49]:
# Verificar se a tradução foi devidamente aplicada
print(df['dia_semana_cadastro'].unique())

['Terça-feira' 'Sexta-feira' 'Quarta-feira' 'Segunda-feira' 'Sábado'
 'Quinta-feira' 'Domingo']


In [50]:
# Mapeamento de meses do ano do inglês para português
meses_ano = {
    'January': 'Janeiro',
    'February': 'Fevereiro',
    'March': 'Março',
    'April': 'Abril',
    'May': 'Maio',
    'June': 'Junho',
    'July': 'Julho',
    'August': 'Agosto',
    'September': 'Setembro',
    'October': 'Outubro',
    'November': 'Novembro',
    'December': 'Dezembro'
}

# Aplica a tradução
df['mes_nome_cadastro'] = df['mes_nome_cadastro'].map(meses_ano)

In [51]:
# Verificar se a tradução foi devidamente aplicada
print(df['mes_nome_cadastro'].unique())

['Janeiro' 'Dezembro' 'Fevereiro' 'Maio' 'Novembro' 'Abril' 'Julho'
 'Março' 'Agosto' 'Setembro' 'Outubro' 'Junho']


### Checagem final das colunas

In [52]:
for col in df.columns:
    print(f"\n🔹 Coluna: {col}")
    valores = df[col].unique()
    print(valores if len(valores) <= 20 else f"{len(valores)} valores únicos. Exibindo os 20 primeiros:")
    print(valores[:20])


🔹 Coluna: id
1981 valores únicos. Exibindo os 20 primeiros:
[ 2795  2285   115 10470  4065 10968  5985  5430  8432   453  1826     1
 10476  1386  5371  7348  4073  1991  4047  9477]

🔹 Coluna: ano_nascimento
57 valores únicos. Exibindo os 20 primeiros:
[1958 1954 1966 1979 1976 1969 1965 1956 1970 1961 1967 1989 1947 1959
 1981 1977 1960 1975 1971 1986]

🔹 Coluna: escolaridade
['Mestrado' 'Doutorado' 'Curso Superior' 'Segundo Grau' 'Primeiro Grau']
['Mestrado' 'Doutorado' 'Curso Superior' 'Segundo Grau' 'Primeiro Grau']

🔹 Coluna: estado_civil
['Solteiro' 'Casado' 'Divorciado']
['Solteiro' 'Casado' 'Divorciado']

🔹 Coluna: salario_anual
1769 valores únicos. Exibindo os 20 primeiros:
[30523. 36634. 43456. 40662. 49544. 57731. 33168. 54450. 35340. 84835.
 57091. 67267. 32474. 21474. 71691. 63564. 44931. 65324. 81044. 62499.]

🔹 Coluna: filhos_em_casa
[2 0 1]
[2 0 1]

🔹 Coluna: adolescentes_em_casa
[1 0 2]
[1 0 2]

🔹 Coluna: data_cadastro
387 valores únicos. Exibindo os 20 primeiros:
<D

### Estatísticas Básicas

In [53]:
# Seleciona colunas numéricas, excluindo 'id' e datas
colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
colunas_numericas = [col for col in colunas_numericas if col != 'id']

# Exibe estatísticas básicas
print(df[colunas_numericas].describe())

       ano_nascimento  salario_anual  filhos_em_casa  adolescentes_em_casa  \
count     1981.000000    1981.000000      1981.00000           1981.000000   
mean      1968.796567   52290.852600         0.44523              0.502776   
std         11.981657   25484.701911         0.53433              0.539938   
min       1893.000000    1730.000000         0.00000              0.000000   
25%       1959.000000   35196.000000         0.00000              0.000000   
50%       1970.000000   51766.000000         0.00000              0.000000   
75%       1977.000000   68281.000000         1.00000              1.000000   
max       1996.000000  666666.000000         2.00000              2.000000   

       dias_desde_ultima_compra  gasto_com_eletronicos  gasto_com_brinquedos  \
count               1981.000000            1981.000000           1981.000000   
mean                  43.647653             304.747097             26.347299   
std                   25.819537             338.425386   

### Detecção de Outliers

In [54]:
outliers = {}

for col in colunas_numericas:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Identifica outliers
    outlier_mask = (df[col] < limite_inferior) | (df[col] > limite_superior)
    outliers[col] = df[col][outlier_mask]

    print(f"\nColuna: {col}")
    print(f"Outliers detectados: {outlier_mask.sum()}")
    print(f"Valores extremos: {df[col][outlier_mask].unique()}")


Coluna: ano_nascimento
Outliers detectados: 2
Valores extremos: [1893 1899]

Coluna: salario_anual
Outliers detectados: 7
Valores extremos: [157146. 160803. 666666. 162397. 157733. 153924. 156924.]

Coluna: filhos_em_casa
Outliers detectados: 0
Valores extremos: []

Coluna: adolescentes_em_casa
Outliers detectados: 0
Valores extremos: []

Coluna: dias_desde_ultima_compra
Outliers detectados: 0
Valores extremos: []

Coluna: gasto_com_eletronicos
Outliers detectados: 33
Valores extremos: [1285. 1248. 1296. 1462. 1478. 1302. 1239. 1349. 1308. 1230. 1332. 1253.
 1394. 1276. 1288. 1493. 1311. 1486. 1492. 1259. 1241. 1245. 1459. 1379.
 1298. 1315. 1396. 1324. 1449.]

Coluna: gasto_com_brinquedos
Outliers detectados: 223
Valores extremos: [104. 130.  80.  82. 174. 169. 107. 105.  81.  96.  86. 193.  83. 148.
 103. 142.  97. 117. 134. 153. 106.  88. 133.  99. 129.  93. 185.  79.
 162. 168.  98. 172. 140. 194.  91. 183. 151. 197. 178. 189. 102. 155.
 115.  90. 114. 199. 154. 123. 108. 137. 120

### Tratamento de Outliers

In [55]:
# Localiza registros com ano de nascimento incoerente
df_outliers_nascimento = df[df['ano_nascimento'] < 1920]
print(df_outliers_nascimento[['id', 'idade', 'ano_nascimento']])

        id  idade  ano_nascimento
513  11004    132            1893
827   1150    126            1899


In [56]:
# Remoção dos registros de idade incoerente
df_sem_nulos = df.shape[0]
df = df.drop(df_outliers_nascimento.index)
df_sem_ano_incoerente = df.shape[0]
print(f'Foram removidas {df_sem_nulos - df_sem_ano_incoerente} linhas com idade incoerente.')

Foram removidas 2 linhas com idade incoerente.


### Salvamento do Dataset Limpo

In [57]:
print(f'Tamanho inicial do DataFrame: {df_inicial} linhas')
print(f'Tamanho final do DataFrame: {df.shape[0]} linhas')
print(f'Total de linhas removidas: {df_inicial - df.shape[0]} linhas')

porcentagem = (df_inicial - df.shape[0]) / df_inicial * 100
print(f'Porcentagem de linhas removidas: {porcentagem: .2f} %')

Tamanho inicial do DataFrame: 2000 linhas
Tamanho final do DataFrame: 1979 linhas
Total de linhas removidas: 21 linhas
Porcentagem de linhas removidas:  1.05 %


In [58]:
# Salvamento condicional
output_path = '/content/drive/MyDrive/Estudos/Dados/Marketing/dados_marketing_limpo.csv'
output_dir = os.path.dirname(output_path)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df.to_csv(output_path, index=False, encoding='utf-8')
print(f'Dataset limpo salvo em: {output_path}')

Dataset limpo salvo em: /content/drive/MyDrive/Estudos/Dados/Marketing/dados_marketing_limpo.csv
